In [1]:
import time
import re
import pandas as pd
import numpy as np
import requests
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle

# 인기영상 크롤링
- 주제에 관계없이 가장 많은 리뷰를 뽑아쓸 수 있는 방법

In [84]:
def get_trending_video():
    url = 'https://www.youtube.com/feed/trending'

    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5)
    
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height

    
    load_box = driver.find_elements_by_css_selector('#grid-container ytd-video-renderer')
    
    info_dict = {'title' : [],
                'link' : [],
                'Youtuber' : [],
                'view' : [],
                'date' : []}
    for video in load_box:
        
        #제목
        title = video.find_element_by_css_selector('#video-title').text
        #url
        link = video.find_element_by_css_selector('#video-title').get_attribute('href')
        #유튜버
        Youtuber = video.find_element_by_css_selector('#text a').text
        #조회수
        view = video.find_element_by_css_selector('#metadata-line span:nth-child(1)').text
        #날짜
        date = video.find_element_by_css_selector('#metadata-line span:nth-child(2)').text
        
        info_dict['title'].append(title)
        info_dict['link'].append(link)
        info_dict['Youtuber'].append(Youtuber)
        info_dict['view'].append(view)
        info_dict['date'].append(date)

    df = pd.DataFrame(info_dict)
    
    return df

In [103]:
df = get_trending_video()

저장할때 날짜정보 저장해서, 하루 전 데이터를 불러오고 url비교해서 크롤링할 url이랑 아닌거랑 구분

입력어 받는 파이썬파일로.  
원하는리뷰수를 인풋?  
-이거계산하는거만들어보고  
url도 인풋으로

In [74]:
#스크롤 횟수

def Youtube_crawl(url, scroll_cnt):
    scroll_cnt = scroll_cnt

    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(2) #댓글창로딩

    body = driver.find_element_by_tag_name('body')

    while scroll_cnt:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        scroll_cnt -= 1

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    print(scroll_cnt, len(reviews))


    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list

### page_down을 사용하면 버벅거리기에 느리다.
- page_down 대신에 scrollHeight 이용

In [ ]:
def Youtube_crawl_2(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    #댓글 최하단까지 이동
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #자세히 보기를 클릭해 리뷰 잘림 현상 방지
    load_more = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_more:
        if load.text == '자세히 보기':
            load.click()

    reviews = driver.find_elements_by_xpath('//*[@id="main"]')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    return df

### 답글이 있는 댓글을 못가져온다

In [2]:
def Youtube_crawl_3(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    #대댓글 불러오기
    load_reply = driver.find_elements_by_css_selector('#contents #text')

    for load in load_reply:
        #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
        if '보기' in load.text: 
            driver.execute_script('arguments[0].click();',load)
            #load.click()의 경우 오류가 발생
        else:
            pass
    
    #대댓글 더 불러오기
    while True:
        try:
            check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

            if len(check_point):
                #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                for load in load_more_reply:
                    driver.execute_script('arguments[0].click();',load)
        except:
            break

    
    #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
    load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_detail:
        if load.text == '자세히 보기':
            driver.execute_script('arguments[0].click();',load)
        else:
            pass

    reviews = driver.find_elements_by_css_selector('#content-text')
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')

    if len(reviews) != len(likes):
        reviews = reviews[:-1] #마지막 review에 Youtube Premium 광고가 섞이는경우가있음

    df = pd.DataFrame(columns=['리뷰', '좋아요'])

    review_list = []
    like_list = []
    for review, like in zip(reviews, likes):
        review_list.append(review.text)
        like_list.append(like.text)

    df['리뷰'] = review_list
    df['좋아요'] = like_list
    
    #좋아요 전처리
    df['좋아요'] = df['좋아요'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨
    
    #좋아요가 없는 칼럼은 1로 대체
    indexes = df[df['좋아요'] == ''].index
    df.loc[indexes, '좋아요'] = 1
    
    return df

Youtube_crawl_3의 경우 리뷰와 좋아요를 따로 뽑고 하나로 합치는 방식이기에, 좋아요와 리뷰의 순서가 맞지 않을 가능성 발생
따라서 각 리뷰가 담긴 박스 전체를 추출하고, 그 안에서 ID, 리뷰, 좋아요, 날짜를 추출하는 방식으로 변경

In [132]:
def Youtube_crawl_4(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩
    
    start = time.time()

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height

    #대댓글 불러오기
    load_reply = driver.find_elements_by_css_selector('#contents #text')

    for load in load_reply:
        #답글을 남기는 '답글' 과 '답글 n개 보기' 중 'n개 보기' 필터링
        if '보기' in load.text: 
            driver.execute_script('arguments[0].click();',load)
            #load.click()의 경우 오류가 발생
        else:
            pass

    #대댓글 더 불러오기
    while True:
        try:
            check_point = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

            if len(check_point):
                #답글이 일정 수준을 넘기면 '답글 더보기'가 등장, 이를 클릭하기 위함
                load_more_reply = driver.find_elements_by_xpath('//*[@id="continuation"]/yt-next-continuation/paper-button/yt-formatted-string')

                for load in load_more_reply:
                    driver.execute_script('arguments[0].click();',load)
        except:
            break
    
    load_content = driver.find_elements_by_xpath('//*[@id="main"]')

    content_info = {'author':[],
                   'date' : [],
                   'review' : [],
                   'like' : []}
  
    for content in load_content:
        try: #Youtube Premium이 걸릴경우가 있다
            #ID
            author = content.find_element_by_css_selector('#author-text span').text
            #날짜
            date = content.find_element_by_css_selector('#header-author  yt-formatted-string  a').text
            #리뷰
            review = content.find_element_by_css_selector('#content-text').text
            #like
            like = content.find_element_by_css_selector('#vote-count-middle').text

            content_info['author'].append(author)
            content_info['date'].append(date)
            content_info['review'].append(review)
            content_info['like'].append(like)
        except:
            pass

    df = pd.DataFrame(content_info)

    #like 전처리
    df['like'] = df['like'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨

    #like가 없는 칼럼은 1로 대체
    indexes = df[df['like'] == ''].index
    df.loc[indexes, 'like'] = 1
    
    end = time.time()
    
    print('time: {}'.format(end-start))
    
    return df

In [30]:
df = Youtube_crawl_4('https://www.youtube.com/watch?v=xwhhAkz-wFk')

,author,date,review,like
0,달님,1일 전,인터스텔라에서 최강의 트롤링을 보여주던 그는 화성에서 살아남기를 찍습니다,127
1,테루시아,1일 전,아빠죽이려한 만박사를\n어떻게든 구하는 머피,105
2,BenYa Entertainment,1일 전,아 그러고보니 배우가....ㄷㄷ,6
3,코끼리,17시간 전(수정됨),우주선에는 캡틴아메리카의 친구와 앤트맨의 친구가 있어서 든든\n나사에서는 닥터스트레...,2
4,창호,3시간 전,맞네 ㅋㅋㅋ,1
...,...,...,...,...
185,조원태,1일 전,저도 다른 결말포함 리뷰어들이 많이 올린 영화는 안 올려주셨으면 하네요,3
186,이시형,23시간 전,@이시경 @ㅐ미없는소리하네 ㅋㅋㅋ,1
187,이시경,22시간 전,야야 미안하다 ㅋㅋ 앞으로는 생각하고 글 싸지를게,1
188,몽초코,18시간 전,@이시경 여러사람한테 욕처먹지 않았으면 넌 그대로였을거아냐 그치? 너같은애들은 변할...,1


대댓글은 영상의 주제와 관계 없이, 리뷰를 남긴 사람에 집중하는 경우가 많다. ex) 서로를 물어뜯거나, 찐이다 등등
따라서 해당 주제에 대한 긍정/부정을 파악하는데에 오히려 도움이 되지 않는다고 판단, 대댓글을 불러오지 않는 크롤러

In [102]:
def Youtube_crawl_without_coco(url):
    url = url

    #자신의 크롬 버전에 맞는 드라이버 다운
    driver = webdriver.Chrome('./chromedriver') #크롬 드라이버 경로
    driver.get(url)

    time.sleep(5) #댓글창로딩

    #모든 댓글 불러오기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        time.sleep(5)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_page_height == last_page_height:
            break

        last_page_height = new_page_height
    
    
    #답글에도 자세히보기를 사용해야할경우가있기때문에 자세히보기 클릭은 답글을 다 연 다음에
    load_detail = driver.find_elements_by_xpath('//*[@id="more"]/span')

    for load in load_detail:
        if load.text == '자세히 보기':
            driver.execute_script('arguments[0].click();',load)
        else:
            pass

    load_content = driver.find_elements_by_xpath('//*[@id="main"]')

    content_info = {
#                     'video_title' : [],
#                     'video_view' : [],
#                     'Youtuber': [],
#                     'video_date' : [],
                    'author':[],
                    'date' : [],
                    'review' : [],
                    'like' : []}
    
    for content in load_content:
        #ID
        author = content.find_element_by_css_selector('#author-text span').text
        #날짜
        date = content.find_element_by_css_selector('#header-author  yt-formatted-string  a').text
        #리뷰
        review = content.find_element_by_css_selector('#content-text').text
        #like
        like = content.find_element_by_css_selector('#vote-count-middle').text

        content_info['author'].append(author)
        content_info['date'].append(date)
        content_info['review'].append(review)
        content_info['like'].append(like)

    df = pd.DataFrame(content_info)
    
#         #title
#     df['video_title'] = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text

#     #view_count
#     df['video_view'] = driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text

#     #Youtuber
#     df['Youtuber'] = driver.find_element_by_css_selector('#text > a').text

#     #date
#     df['date'] = driver.find_element_by_css_selector('#date > yt-formatted-string').text

#     #like 전처리
#     df['like'] = df['like'].apply(lambda x: re.sub('천 | 만','000',x)) #천의 경우는 1천 / 만의 경우는 1.1만 의 경우로 표현됨

    #like가 없는 칼럼은 1로 대체
    indexes = df[df['like'] == ''].index
    df.loc[indexes, 'like'] = 1
    
    return df

In [101]:
df = Youtube_crawl_without_coco('https://www.youtube.com/watch?v=HhgcXuN0MlU&t=599s')
df.to_pickle('유튜브 리뷰 댓글제거.pkl')

ValueError: arrays must all be same length

보다 많은 리뷰가 성공적으로 불러와졌다


reviews = driver.find_elements_by_xpath('//*[@id="main"]')
reviews = driver.find_elements_by_css_selector('#content-text')

---
xpath로 가져오는 데이터와 css_selector로 가져오는 데이터가 서로 다르다
xpath, css_selector로 가져왔을 때 뭔가 불만족스럽다면 서로를 바꿔보자

### 좋아요 수 전처리
- '1천', or ''

답글과 자세히보기문제

like에 가중치를 둔다면?
댓글 * like로 가중치를두어계산(이때 log2 씌워주고) #like가없다면 차이를두어야겠지?